# This kernel is U-Net Baseline written by PyTorch
In this kernel, there are many places that are simplified now.  
So, you should fix these bad points.  

[U-Net web site](https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/)  
[U-Net paper](https://arxiv.org/abs/1505.04597)  

I reference [this blog post](https://lp-tech.net/articles/hzfn7?page=2  ) in U-Net installation.  
Thank you awesome this blog post.  

This is [my EDA](https://www.kaggle.com/go1dfish/fgvc6-simple-eda).  
If you don't know this competition rule and data, this EDA might help you.  

# Import modules

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("../input"))
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook as tqdm

import torch
import torch.nn as nn
from torch import optim
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.autograd import Function, Variable
from pathlib import Path
from itertools import groupby

['train.csv', 'train', 'label_descriptions.json', 'test', 'sample_submission.csv']


In [2]:
input_dir = "../input/"
train_img_dir = "../input/train/"
test_img_dir = "../input/test/"

WIDTH = 512
HEIGHT = 512
category_num = 46 + 1

ratio = 8

epoch_num = 8
batch_size = 4

device = "cuda:0"

In [3]:
len(os.listdir("../input/train/"))

45195

In [4]:
len(os.listdir("../input/test/"))

3193

In [5]:
train_df = pd.read_csv(input_dir + "train.csv")
train_df.head()

,ImageId,EncodedPixels,Height,Width,ClassId
0,00000663ed1ff0c4e0132b9b9ac53f6e.jpg,6068157 7 6073371 20 6078584 34 6083797 48 608...,5214,3676,6
1,00000663ed1ff0c4e0132b9b9ac53f6e.jpg,6323163 11 6328356 32 6333549 53 6338742 75 63...,5214,3676,0
2,00000663ed1ff0c4e0132b9b9ac53f6e.jpg,8521389 10 8526585 30 8531789 42 8537002 46 85...,5214,3676,28
3,00000663ed1ff0c4e0132b9b9ac53f6e.jpg,12903854 2 12909064 7 12914275 10 12919485 15 ...,5214,3676,31
4,00000663ed1ff0c4e0132b9b9ac53f6e.jpg,10837337 5 10842542 14 10847746 24 10852951 33...,5214,3676,32


In [6]:
train_df.shape

(331213, 5)

# Define utils
For simplicity, It focus only category

In [7]:
def make_onehot_vec(x):
    vec = np.zeros(category_num)
    vec[x] = 1
    return vec

In [8]:
def make_mask_img(segment_df):
    seg_width = segment_df.at[0, "Width"]
    seg_height = segment_df.at[0, "Height"]
    seg_img = np.full(seg_width*seg_height, category_num-1, dtype=np.int32)
    for encoded_pixels, class_id in zip(segment_df["EncodedPixels"].values, segment_df["ClassId"].values):
        pixel_list = list(map(int, encoded_pixels.split(" ")))
        for i in range(0, len(pixel_list), 2):
            start_index = pixel_list[i] - 1
            index_len = pixel_list[i+1] - 1
            seg_img[start_index:start_index+index_len] = int(class_id.split("_")[0])
    seg_img = seg_img.reshape((seg_height, seg_width), order='F')
    seg_img = cv2.resize(seg_img, (WIDTH, HEIGHT), interpolation=cv2.INTER_NEAREST)
    """
    seg_img_onehot = np.zeros((HEIGHT, WIDTH, category_num), dtype=np.int32)
    #seg_img_onehot = np.zeros((seg_height//ratio, seg_width//ratio, category_num), dtype=np.int32)
    # OPTIMIZE: slow
    for ind in range(HEIGHT):
        for col in range(WIDTH):
            seg_img_onehot[ind, col] = make_onehot_vec(seg_img[ind, col])
    """
    return seg_img

In [9]:
def train_generator(df, batch_size):
    img_ind_num = df.groupby("ImageId")["ClassId"].count()
    index = df.index.values[0]
    trn_images = []
    seg_images = []
    for i, (img_name, ind_num) in enumerate(img_ind_num.items()):
        img = cv2.imread(train_img_dir + img_name)
        img = cv2.resize(img, (WIDTH, HEIGHT), interpolation=cv2.INTER_AREA)
        segment_df = (df.loc[index:index+ind_num-1, :]).reset_index(drop=True)
        index += ind_num
        if segment_df["ImageId"].nunique() != 1:
            raise Exception("Index Range Error")
        seg_img = make_mask_img(segment_df)
        
        # HWC -> CHW
        img = img.transpose((2, 0, 1))
        #seg_img = seg_img.transpose((2, 0, 1))
        
        trn_images.append(img)
        seg_images.append(seg_img)
        if((i+1) % batch_size == 0):
            yield np.array(trn_images, dtype=np.float32) / 255, np.array(seg_images, dtype=np.int32)
            trn_images = []
            seg_images = []
    if(len(trn_images) != 0):
        yield np.array(trn_images, dtype=np.float32) / 255, np.array(seg_images, dtype=np.int32)

In [10]:
def test_generator(df):
    img_names = df["ImageId"].values
    for img_name in img_names:
        img = cv2.imread(test_img_dir + img_name)
        img = cv2.resize(img, (WIDTH, HEIGHT), interpolation=cv2.INTER_AREA)
        # HWC -> CHW
        img = img.transpose((2, 0, 1))
        yield img_name, np.asarray([img], dtype=np.float32) / 255

In [11]:
def encode(input_string):
    return [(len(list(g)), k) for k,g in groupby(input_string)]

def run_length(label_vec):
    encode_list = encode(label_vec)
    index = 1
    class_dict = {}
    for i in encode_list:
        if i[1] != category_num-1:
            if i[1] not in class_dict.keys():
                class_dict[i[1]] = []
            class_dict[i[1]] = class_dict[i[1]] + [index, i[0]]
        index += i[0]
    return class_dict

# Define Network

In [12]:
class double_conv(nn.Module):
    '''(conv => BN => ReLU) * 2'''
    def __init__(self, in_ch, out_ch):
        super(double_conv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        x = self.conv(x)
        return x


class inconv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(inconv, self).__init__()
        self.conv = double_conv(in_ch, out_ch)

    def forward(self, x):
        x = self.conv(x)
        return x


class down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(down, self).__init__()
        self.mpconv = nn.Sequential(
            nn.MaxPool2d(2),
            double_conv(in_ch, out_ch)
        )

    def forward(self, x):
        x = self.mpconv(x)
        return x


class up(nn.Module):
    def __init__(self, in_ch, out_ch, bilinear=True):
        super(up, self).__init__()

        #  would be a nice idea if the upsampling could be learned too,
        #  but my machine do not have enough memory to handle all those weights
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_ch//2, in_ch//2, 2, stride=2)

        self.conv = double_conv(in_ch, out_ch)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        diffX = x1.size()[2] - x2.size()[2]
        diffY = x1.size()[3] - x2.size()[3]
        x2 = F.pad(x2, (diffX // 2, int(diffX / 2),
                        diffY // 2, int(diffY / 2)))
        x = torch.cat([x2, x1], dim=1)
        x = self.conv(x)
        return x


class outconv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(outconv, self).__init__()
        self.conv = nn.Conv2d(in_ch, out_ch, 1)

    def forward(self, x):
        x = self.conv(x)
        return x

    
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes):
        super(UNet, self).__init__()
        self.inc = inconv(n_channels, 64)
        self.down1 = down(64, 128)
        self.down2 = down(128, 256)
        self.down3 = down(256, 512)
        self.down4 = down(512, 512)
        self.up1 = up(1024, 256)
        self.up2 = up(512, 128)
        self.up3 = up(256, 64)
        self.up4 = up(128, 64)
        self.outc = outconv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        return x

# Training

In [13]:
train_df.shape

(331213, 5)

In [14]:
333415 // 4

83353

In [15]:
train_df.iloc[83348:83354, :]

,ImageId,EncodedPixels,Height,Width,ClassId
83348,404b0d1b58a1af0841e1e46f975118eb.jpg,337743 2 338739 6 339735 10 340732 13 341728 1...,1000,667,22
83349,404b0d1b58a1af0841e1e46f975118eb.jpg,235561 2 236558 5 237554 9 238551 13 239549 15...,1000,667,8
83350,404b0d1b58a1af0841e1e46f975118eb.jpg,219425 10 220411 30 221398 50 222385 68 223374...,1000,667,0
83351,404b0d1b58a1af0841e1e46f975118eb.jpg,298226 2 299225 7 300225 11 301225 14 302224 1...,1000,667,28
83352,404b0d1b58a1af0841e1e46f975118eb.jpg,379364 12 380342 1 380352 27 381340 4 381351 3...,1000,667,31
83353,404b0d1b58a1af0841e1e46f975118eb.jpg,219425 10 220411 30 221398 50 222385 68 223374...,1000,667,31


In [16]:
train_df.iloc[73350:73354, :]

,ImageId,EncodedPixels,Height,Width,ClassId
73350,3892703cf1e7f6a3cc1892fcb3daedae.jpg,85434 2 88642 6 91851 9 95059 14 98268 17 1014...,3211,2141,31
73351,3892703cf1e7f6a3cc1892fcb3daedae.jpg,82220 2 85428 6 88637 9 91846 13 95055 16 9826...,3211,2141,1
73352,3893868714043964534a6cdbaf1f0d75.jpg,7536415 56 7542028 60 7547642 62 7553255 65 75...,5616,3744,23
73353,3893868714043964534a6cdbaf1f0d75.jpg,12534531 11 12540144 33 12545757 55 12551369 6...,5616,3744,23


For simplicity, use about 25% data.  

In [17]:
net = UNet(n_channels=3, n_classes=category_num).to(device)

optimizer = optim.SGD(
    net.parameters(),
    lr=0.1,
    momentum=0.9,
    weight_decay=0.0005
)

criterion = nn.CrossEntropyLoss()

In [18]:
val_sta = 73352
val_end = 83351
train_loss = []
valid_loss = []
for epoch in range(epoch_num):
    epoch_trn_loss = 0
    train_len = 0
    net.train()
    for iteration, (X_trn, Y_trn) in enumerate(tqdm(train_generator(train_df.iloc[:val_sta, :], batch_size))):
        X = torch.tensor(X_trn, dtype=torch.float32).to(device)
        Y = torch.tensor(Y_trn, dtype=torch.long).to(device)
        train_len += len(X)
        
        #Y_flat = Y.view(-1)
        mask_pred = net(X)
        #mask_prob = torch.softmax(mask_pred, dim=1)
        #mask_prob_flat = mask_prob.view(-1)
        loss = criterion(mask_pred, Y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_trn_loss += loss.item()
        
        if iteration % 100 == 0:
            print("train loss in {:0>2}epoch  /{:>5}iter:    {:<10.8}".format(epoch+1, iteration, epoch_trn_loss/(iteration+1)))
        
    train_loss.append(epoch_trn_loss/(iteration+1))
    print("train {}epoch loss({}iteration):    {:10.8}".format(epoch+1, iteration, train_loss[-1]))
    
    epoch_val_loss = 0
    val_len = 0
    net.eval()
    for iteration, (X_val, Y_val) in enumerate(tqdm(train_generator(train_df.iloc[val_sta:val_end, :], batch_size))):
        X = torch.tensor(X_val, dtype=torch.float32).to(device)
        Y = torch.tensor(Y_val, dtype=torch.long).to(device)
        val_len += len(X)
        
        #Y_flat = Y.view(-1)
        
        mask_pred = net(X)
        #mask_prob = torch.softmax(mask_pred, dim=1)
        #mask_prob_flat = mask_prob.view(-1)
        loss = criterion(mask_pred, Y)
        epoch_val_loss += loss.item()
        
        if iteration % 100 == 0:
            print("valid loss in {:0>2}epoch  /{:>5}iter:    {:<10.8}".format(epoch+1, iteration, epoch_val_loss/(iteration+1)))
        
    valid_loss.append(epoch_val_loss/(iteration+1))
    print("valid {}epoch loss({}iteration):    {:10.8}".format(epoch+1, iteration, valid_loss[-1]))

/opt/conda/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


train loss in 01epoch  /    0iter:    3.7808681 
train loss in 01epoch  /  100iter:    1.2731448 
train loss in 01epoch  /  200iter:    1.1407036 
train loss in 01epoch  /  300iter:    1.1156803 
train loss in 01epoch  /  400iter:    1.0912886 
train loss in 01epoch  /  500iter:    1.0671572 
train loss in 01epoch  /  600iter:    1.0548593 
train loss in 01epoch  /  700iter:    1.0494163 
train loss in 01epoch  /  800iter:    1.0420111 
train loss in 01epoch  /  900iter:    1.0390621 
train loss in 01epoch  / 1000iter:    1.0276598 
train loss in 01epoch  / 1100iter:    1.0224457 
train loss in 01epoch  / 1200iter:    1.0185198 
train loss in 01epoch  / 1300iter:    1.0184505 
train loss in 01epoch  / 1400iter:    1.0162778 
train loss in 01epoch  / 1500iter:    1.0127839 
train loss in 01epoch  / 1600iter:    1.0096741 
train loss in 01epoch  / 1700iter:    1.0068054 
train loss in 01epoch  / 1800iter:    1.0027825 
train loss in 01epoch  / 1900iter:    1.0022021 
train loss in 01epoc

valid loss in 01epoch  /    0iter:    0.99603325
valid loss in 01epoch  /  100iter:    0.95233545
valid loss in 01epoch  /  200iter:    0.95109498
valid loss in 01epoch  /  300iter:    0.94471123

valid 1epoch loss(347iteration):    0.93913579


train loss in 02epoch  /    0iter:    1.1434746 
train loss in 02epoch  /  100iter:    0.95416942
train loss in 02epoch  /  200iter:    0.91928346
train loss in 02epoch  /  300iter:    0.92926362
train loss in 02epoch  /  400iter:    0.92423466
train loss in 02epoch  /  500iter:    0.91528389
train loss in 02epoch  /  600iter:    0.91618777
train loss in 02epoch  /  700iter:    0.92091957
train loss in 02epoch  /  800iter:    0.92140739
train loss in 02epoch  /  900iter:    0.92528268
train loss in 02epoch  / 1000iter:    0.91893268
train loss in 02epoch  / 1100iter:    0.91874149
train loss in 02epoch  / 1200iter:    0.91952493
train loss in 02epoch  / 1300iter:    0.92368216
train loss in 02epoch  / 1400iter:    0.92462523
train loss in 02epoch  / 1500iter:    0.9236668 
train loss in 02epoch  / 1600iter:    0.9230057 
train loss in 02epoch  / 1700iter:    0.92280991
train loss in 02epoch  / 1800iter:    0.92132775
train loss in 02epoch  / 1900iter:    0.922619  
train loss in 02epoc

valid loss in 02epoch  /    0iter:    1.0323303 
valid loss in 02epoch  /  100iter:    0.97016995
valid loss in 02epoch  /  200iter:    0.96665149
valid loss in 02epoch  /  300iter:    0.96176101

valid 2epoch loss(347iteration):    0.95533126


...

valid loss in 06epoch  /    0iter:    0.75528651
valid loss in 06epoch  /  100iter:    0.80938272
valid loss in 06epoch  /  200iter:    0.81343751
valid loss in 06epoch  /  300iter:    0.80768086

valid 6epoch loss(347iteration):    0.80217946


train loss in 07epoch  /    0iter:    0.81937146
train loss in 07epoch  /  100iter:    0.76041588
train loss in 07epoch  /  200iter:    0.73644228
train loss in 07epoch  /  300iter:    0.7529125 
train loss in 07epoch  /  400iter:    0.75047421
train loss in 07epoch  /  500iter:    0.74310587
train loss in 07epoch  /  600iter:    0.74382038
train loss in 07epoch  /  700iter:    0.74776025
train loss in 07epoch  /  800iter:    0.74911626
train loss in 07epoch  /  900iter:    0.7526035 
train loss in 07epoch  / 1000iter:    0.74741268
train loss in 07epoch  / 1100iter:    0.74697296
train loss in 07epoch  / 1200iter:    0.74718113
train loss in 07epoch  / 1300iter:    0.75096219
train loss in 07epoch  / 1400iter:    0.75250301
train loss in 07epoch  / 1500iter:    0.75143292
train loss in 07epoch  / 1600iter:    0.7517867 
train loss in 07epoch  / 1700iter:    0.7523281 
train loss in 07epoch  / 1800iter:    0.75091743
train loss in 07epoch  / 1900iter:    0.7528693 
train loss in 07epoc

valid loss in 07epoch  /    0iter:    0.83411443
valid loss in 07epoch  /  100iter:    0.81385857
valid loss in 07epoch  /  200iter:    0.81966688
valid loss in 07epoch  /  300iter:    0.81518302

valid 7epoch loss(347iteration):    0.80929136


train loss in 08epoch  /    0iter:    0.84060639
train loss in 08epoch  /  100iter:    0.74190513
train loss in 08epoch  /  200iter:    0.7182923 
train loss in 08epoch  /  300iter:    0.73472241
train loss in 08epoch  /  400iter:    0.73045697
train loss in 08epoch  /  500iter:    0.72378685
train loss in 08epoch  /  600iter:    0.72711251
train loss in 08epoch  /  700iter:    0.73225973
train loss in 08epoch  /  800iter:    0.73342164
train loss in 08epoch  /  900iter:    0.73868762
train loss in 08epoch  / 1000iter:    0.73437165
train loss in 08epoch  / 1100iter:    0.73351673
train loss in 08epoch  / 1200iter:    0.73457308
train loss in 08epoch  / 1300iter:    0.73843175
train loss in 08epoch  / 1400iter:    0.74094191
train loss in 08epoch  / 1500iter:    0.74047159
train loss in 08epoch  / 1600iter:    0.74063366
train loss in 08epoch  / 1700iter:    0.74148685
train loss in 08epoch  / 1800iter:    0.74039576
train loss in 08epoch  / 1900iter:    0.74231388
train loss in 08epoc

KeyboardInterrupt: 

In [ ]:
#plt.plot(list(range(epoch_num)), train_loss, color='green')
#plt.plot(list(range(epoch_num)), valid_loss, color='blue')

In [19]:
!pwd

/kaggle/working


In [21]:
torch.save(net.state_dict(), 'Unet_model')

# Test

In [ ]:
sample_df = pd.read_csv(input_dir + "sample_submission.csv")

In [ ]:
import torch
import gc
for obj in gc.get_objects():
    try:
        if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
            print(type(obj), obj.size())
    except:
        pass

In [ ]:
sub_list = []
net.eval()
for img_name, img in test_generator(sample_df):
    X = torch.tensor(img, dtype=torch.float32).to(device)
    mask_pred = net(X)
    mask_pred = mask_pred.cpu().detach().numpy()
    mask_prob = np.argmax(mask_pred, axis=1)
    mask_prob = mask_prob.ravel(order='F')
    class_dict = run_length(mask_prob)
    if len(class_dict) == 0:
        sub_list.append([img_name, "1 1", 1])
    else:
        for key, val in class_dict.items():
            sub_list.append([img_name, " ".join(map(str, val)), key])

# Make Submission File

In [ ]:
submission_df = pd.DataFrame(sub_list, columns=sample_df.columns.values)

In [ ]:
submission_df

In [ ]:
submission_df.to_csv("submission.csv", index=False)

# Thank you for watching!
Please tell me when I make mistakes in program and English.  
I hope this kernel will help.  
If you think this kernel is useful, please upvote. If you do, I feel happy and get enough sleep.  